In [ ]:
#creating spark instance
from pyspark.sql import SparkSession
spark = SparkSession .builder.appName('datacleaningrevision').getOrCreate()

In [ ]:
#import libraries
import pandas as pd
import numpy as np

In [ ]:
from pyspark.sql import functions as sf

In [ ]:
#Read column names
#headers = spark.read.option("sep", "\t").csv("..\column_headers.tsv",inferSchema =True, header=True)
#headers.printSchema()

In [ ]:
#Read column names
headers1 = spark.read.csv("sample1.csv",inferSchema =True, header=True)
#headers1.printSchema()

In [ ]:
#Read data from tsv file
rows = spark.read.option("sep", "\t").csv("..\hit_data.tsv",inferSchema =True, header=False)
#rows.printSchema()

In [ ]:
#adding header to data
rowsWithHeader1 = headers1.union(rows)
rowsWithHeader1.printSchema()

In [ ]:
rowsWithHeader1.count()

In [ ]:
#types = [f.dataType for f in rows.schema.fields]
#sn = headers.schema.names

In [ ]:
#adding header to data
#rowsWithHeader = headers.union(rows)
#rowsWithHeader.dtypes
#rowsCount = rowsWithHeader.count()
#rowsCount

In [ ]:
#filtering rows based on exclude_hit column value. Removes all rows whose exclude_hit values less are <= 0
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.exclude_hit<=0)

In [ ]:
#Print count
rowsCount = rowsWithHeader1.count()
rowsCount

In [ ]:
#filtering rows based on hit_source column value. Exclude all rows with hit_source = 5,7,8,9.
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=5)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=7)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=8)
rowsWithHeader1 = rowsWithHeader1.filter(rowsWithHeader1.hit_source!=9)

In [ ]:
#Selecting only post columns
postData = rowsWithHeader1.select(*filter(lambda col: 'post_' in col,rowsWithHeader1.columns))

In [ ]:
#Printing no. of columns
len(postData.columns)

In [ ]:
#Print count
postRowsCount = postData.count()
postRowsCount

In [ ]:
#Add a new column which conains values to idenify unique visitors
MergedColumns = postData.withColumn('joined_column', sf.concat(sf.col('post_visid_high'),sf.lit(''), sf.col('post_visid_low')))

In [ ]:
#Renaming column in dataframe
MergedColumns=MergedColumns.withColumnRenamed('joined_column', 'post_uniqueId')

In [ ]:
#Select the column post_uniqueID
UniqueId=MergedColumns.select('post_uniqueId')
#Print count
UniqueId.count()

In [ ]:
#Removing post_mobile columns from dataframe
postDataWithoutMobile = MergedColumns.drop(*filter(lambda col: 'mobile' in col,postData.columns))
#Printing no. of columns
len(postDataWithoutMobile.columns)

In [ ]:
#Print count
postDataWithoutMobile.count()

In [ ]:
#Get column names
columnNames = postDataWithoutMobile.schema.names
#print column names
#columnNames
#type(columnNames)

In [ ]:
#Takes more time to compute as it has to iterate through each row for each column in columnNames
#nonNullColumns = []
#for name in columnNames:
#    if(postData.where(postDataWithoutMobile[name].isNotNull()).count() == postRowsCount):
#        nonNullColumns.append(name)

In [ ]:
#nonNullColumns

In [ ]:
#import pyspark.sql.functions as func

#for col in postData.columns:
#    if (postData.filter(func.col(col).isNull()).count() == postData.select(func.col(col)).count()):
#        postData = postData.drop(col)
#    else:
#        print('did not drop' + col )

In [ ]:
#Display post_referrer column
postDataWithoutMobile.select('post_referrer').show()

In [ ]:
#select users with cart in referrer
cartRows = postDataWithoutMobile.filter(postDataWithoutMobile.post_referrer.rlike("^(.)*(cart|CART|Cart)(.)*+$"))

In [ ]:
#Print count
cartRows.count()

In [ ]:
#list(cartRows)
#cartRows.select('post_referrer').show()

In [ ]:
#storing all existing distinct users
allids = [list(x.asDict().values())[0] for x in cartRows.select("post_uniqueid").distinct().collect()]

In [ ]:
#Print count
print("{}".format(len(allids)))

In [ ]:
#loop through and print all rows of ids
#for x in allids[0:3]:
#    postDataWithoutMobile.where(postDataWithoutMobile.post_uniqueId == x).show(5,False)

In [ ]:
#Converting to pandas dataframe
cartPDF = cartRows.toPandas()
cartPDF.shape

In [ ]:
#combining post_visid_high with post_visid_low and storing the value in new column newuid.
#which is used to identify unique visitor.
#cartPDF['newuid'] = cartPDF.post_visid_high.astype(str) + cartPDF.post_visid_low.astype(str)
#cartPDF.shape

In [ ]:
#cartPDF['newuid']

In [ ]:
#Removes any columns with null values
cartRowsWithoutNullColumns = cartPDF.dropna(axis=1,how='all')
cartRowsWithoutNullColumns.shape

In [ ]:
#Get distinct values of post_uniqueId column
uniqueCartUsers = cartPDF.post_uniqueId.unique()
len(uniqueCartUsers)

In [ ]:
#Regex to find event list having 1 at beginning, in middle and at end.
event1Rows = postDataWithoutMobile.filter(postDataWithoutMobile.post_event_list.rlike("^(1,[0-9 , . =]*|[0-9 , . =]*,1,[0-9 , . =]*|[0-9 , . =]*,1)$"))

In [ ]:
#Displays top 20 rows
event1Rows.show()

In [ ]:
event1Rows.select("post_event_list","post_uniqueId").show()

In [ ]:
event1Rows.count()

In [ ]:
#convert to pandas dataframe
pandasDF = event1Rows.toPandas()

In [ ]:
pandasDF

In [ ]:
pandasDF.shape

In [ ]:
#combining post_visid_high with post_visid_low and storing the value in new column newuid.
#which is used to identify unique visitor. post_uniqueId has already been added.
#pandasDF['newuid'] = pandasDF.post_visid_high + pandasDF.post_visid_low

In [ ]:
pandasDF.dtypes

In [ ]:
event1Count = len(pandasDF.index)

In [ ]:
event1Count

In [ ]:
#Removes any columns with null values
pandasDFWithoutNullColumns = pandasDF.dropna(axis=1,how='all')

In [ ]:
pandasDFWithoutNullColumns.shape

In [ ]:
pandasDFWithoutNullColumnsNames = list(pandasDFWithoutNullColumns)

In [ ]:
pandasDFWithoutNullColumnsNames

In [ ]:
#Drops duplicate columns and also drops null columns
pandasDFWithoutDuplicateColumns = pandasDFWithoutNullColumns.T.drop_duplicates().T

In [ ]:
pandasDFWithoutDuplicateColumnsNames = list(pandasDFWithoutDuplicateColumns)

In [ ]:
removedDuplicateColumns = list(set(pandasDFWithoutNullColumnsNames) - set(pandasDFWithoutDuplicateColumnsNames))

In [ ]:
removedDuplicateColumns

In [ ]:
pandasDFWithoutDuplicateColumns.shape

In [ ]:
list(pandasDFWithoutDuplicateColumns)

In [ ]:
pandasDFWithoutDuplicateColumns

In [ ]:
result1 = pandasDFWithoutDuplicateColumns.dropna(axis=1,thresh=event1Count/2)

In [ ]:
result1.shape

In [ ]:
list(result1)

In [ ]:
result1['post_referrer']